In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=1000)
L=LaurentSeriesRing(QQ,'x', default_prec=1000)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=1000)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.

def uniteList(lst):
    ln=len(lst)
    if ln<2:return lst
    answer=[]
    for j in [0..ln-1]:
        lj=lst[j]
        if (lj in answer)==False:
            answer=answer+[lj]
    return answer

def delete_element(elt,ls):
    new=[]
    for j in ls:
        if j!=elt:new=new+[j]
    return new

def sortAndSmash(lst): #sorts and deletes repeats
    cropped=lst
    new=[]
    while len(cropped)>0:
        m=min(cropped)
        new=new+[m]
        cropped=delete_element(m,cropped)
    return(new)

def difference_table(lst):
    diffs=[]
    for k in [0..len(lst)-2]:
        diff=lst[k+1]-lst[k]
        diffs=diffs+[diff]
    return diffs

def evaluate(expression,incomingvalue):
    expr=str(expression)
    output=sage_eval(expr,locals={'x':incomingvalue})
    return output

P=Primes()

def prime(n): return P.unrank(n-1)

def unite(A,B):
    return set(A).union(B)

def factor_integer(n): #this is like "factor(integer)", but
    #the bases and exponents can be easily extracted.
    F=factor(n)
    return list(F)

def orderInteger(p,n):
    if n!=0:
        lst=n.digits(p)
        count=0
        k=0
        while lst[k]==0:
            count=count+1
            k=k+1
        return count
    if n==0:return(Infinity)

def order(p,fr):
    nn=numerator(QQ(fr))
    dn=denominator(QQ(fr))
    return orderInteger(p,nn)-orderInteger(p,dn)

def digitsum(p,n):return add(n.digits(p))

def digitsum(p,n):
    return sum(n.digits(p))

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])

def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer

def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))

def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))

def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)

def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))

def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)

def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)

def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # 
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.

def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)

def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]

def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)

def j(n,m):return L(xjNew(n,m)/x)

def xjLeoNew(n,m,multiplier):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(multiplier)**i
        return rt
    return xjNewStep(n-1,m)

def jLeo(n,m,multiplier):return L(xjLeoNew(n,m,multiplier)/x)

def JpowerConstantTerm(m,power):
    trun=J(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def jpowerConstantTerm(m,power):
    trun=j(power,m)
    trun=expand(trun^power)
    polyversion=expand(trun*x^power)
    constant=polynomialCoefficient(power,polyversion)
    return constant

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    
    return polynomialCoefficient(dg,polynomial)

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)

def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())

def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr

def EulerMultiplication(listo):
    # Lemma 2.11 from my 1998 Experimental Mathematics paper.
    # Here the set A in the lemma is {1, 2, ...} = Z^+.
    p_A=[1] # so p_A[0]=1 as on top of p. 266, column 2.
    def ex(n):return listo[n-1] # ex is the exponent on the factor 1-x^n: -f(n)/n.
    def f(n):return -n*ex(n)
    answer=[1]
    def f_A(k):
        summ=0
        dk=divisors(k)
        for d in dk: #Here is where A = Z^+ is used to drop a constraint on k.
            summ=summ+f(d)
        return summ
    for n in [1..len(listo)-1]:
        sm=0
        for k in [1..n]:
            sm=sm+f_A(k)*p_A[n-k]
        p_A=p_A+[sm/n]
    return p_A

def listToPoly(listo):
    lenth=len(listo)
    answer=0
    for k in [1..lenth]:
        cf=listo[k-1]
        answer=answer+cf*x^(k-1)
    return answer
    
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

print("okay")

okay


In [8]:
for n in [1..5]:
    print((n,sigma(n,k=11)))

(1, 1)
(2, 2049)
(3, 177148)
(4, 4196353)
(5, 48828126)


In [2]:
import random
import time


def specialModulus(a):# I alter this definition, relative to the previous cell.
    ds=digitsum(2,a)
    return 2*8^ds

def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    mlist=[]
    for a in [1..n+1]:
        mlist=mlist+[specialModulus(a)]
    sm=max(mlist) # probably bc max(mlist) = lcm(mlist) 
    for a in [1..n+1]:
        r=random.randint(-10, 10)
        d=order(2,a)
        oddpart=a/2^d
        #ord(2,tau(2^d))= (I believe) 3d
        # so ord(2,tau(purepart))=ord(2,tau(2^d)) = 3d
        # so tau(purepart) = 2^(3d)
        # so tau(a) = tau(purepart)*tau(oddpart)=
        # 8^d*tau(oddpart); then we replace tau(oddpart)
        # with sigma(oddpart,k=11) by Ramanujan's congruence
        term=8^d*sigma(oddpart,k=11)+r*sm
        ans=ans+term*x^a
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=1000)

no=[]
for b in [1..5]:
    print()
    ls=[]
    for a in [1..500]: # 100 < precision is necessary
        start=time.time()
        c=p2RecipPowerConstantTerm(a)
        o2=order(2,c)
        ds=digitsum(2,a)
        test=o2-3*ds
        finish=time.time()
        print((b,a,test,finish-start))
        if test!=0:ls=ls+[a]
    print()
    print("bad a's for trial ",b);print();print((ls))
    no=no+[(b,len(ls))]
print("------------------------------------------------------------------")
print("no:");print();print(no)


(1, 1, 0, 0.030900001525878906)
(1, 2, 0, 0.07921671867370605)
(1, 3, 0, 0.07221102714538574)
(1, 4, 0, 0.0865321159362793)
(1, 5, 0, 0.07135009765625)
(1, 6, 0, 0.13343286514282227)
(1, 7, 0, 0.11745095252990723)
(1, 8, 0, 0.10263228416442871)
(1, 9, 0, 0.09354305267333984)
(1, 10, 0, 0.08822178840637207)
(1, 11, 0, 0.11419081687927246)
(1, 12, 0, 0.10160207748413086)
(1, 13, 0, 0.13862180709838867)
(1, 14, 0, 0.12421512603759766)
(1, 15, 0, 0.1482069492340088)
(1, 16, 0, 0.17871475219726562)
(1, 17, 0, 0.16344404220581055)
(1, 18, 0, 0.1792900562286377)
(1, 19, 0, 0.15658187866210938)
(1, 20, 0, 0.17223000526428223)
(1, 21, 0, 0.139312744140625)
(1, 22, 0, 0.17194318771362305)
(1, 23, 0, 0.17847394943237305)
(1, 24, 0, 0.18164300918579102)
(1, 25, 0, 0.16023993492126465)
(1, 26, 0, 0.18455076217651367)
(1, 27, 0, 0.17637205123901367)
(1, 28, 0, 0.1604452133178711)
(1, 29, 0, 0.17949700355529785)
(1, 30, 0, 0.21554017066955566)
(1, 31, 0, 0.21610379219055176)
(1, 32, 0, 0.19489407539

(1, 257, 0, 0.5261588096618652)
(1, 258, 0, 0.5312988758087158)
(1, 259, 0, 0.5694267749786377)
(1, 260, 0, 0.5171530246734619)
(1, 261, 0, 0.5585129261016846)
(1, 262, 0, 0.5429871082305908)
(1, 263, 0, 0.5052511692047119)
(1, 264, 0, 0.46723127365112305)
(1, 265, 0, 0.15713787078857422)
(1, 266, 0, 0.5157699584960938)
(1, 267, 0, 0.5592148303985596)
(1, 268, 0, 0.5383038520812988)
(1, 269, 0, 0.5738639831542969)
(1, 270, 0, 0.5845050811767578)
(1, 271, 0, 0.5480680465698242)
(1, 272, 0, 0.1679389476776123)
(1, 273, 0, 0.5136399269104004)
(1, 274, 0, 0.536431074142456)
(1, 275, 0, 0.5532710552215576)
(1, 276, 0, 0.557488203048706)
(1, 277, 0, 0.522691011428833)
(1, 278, 0, 0.6356449127197266)
(1, 279, 0, 0.6459259986877441)
(1, 280, 0, 0.6043450832366943)
(1, 281, 0, 0.5758051872253418)
(1, 282, 0, 0.613461971282959)
(1, 283, 0, 0.5721909999847412)
(1, 284, 0, 0.550753116607666)
(1, 285, 0, 0.6263589859008789)
(1, 286, 0, 0.6441740989685059)
(1, 287, 0, 0.5871748924255371)
(1, 288, 0,

(2, 14, 0, 0.16825199127197266)
(2, 15, 0, 0.17241430282592773)
(2, 16, 0, 0.12387204170227051)
(2, 17, 0, 0.032285213470458984)
(2, 18, 0, 0.11695599555969238)
(2, 19, 0, 0.16144800186157227)
(2, 20, 0, 0.16128802299499512)
(2, 21, 0, 0.12122631072998047)
(2, 22, 0, 0.1701221466064453)
(2, 23, 0, 0.1593480110168457)
(2, 24, 0, 0.1712026596069336)
(2, 25, 0, 0.17498087882995605)
(2, 26, 0, 0.1439499855041504)
(2, 27, 0, 0.17733311653137207)
(2, 28, 0, 0.152174711227417)
(2, 29, 0, 0.15683603286743164)
(2, 30, 0, 0.2085421085357666)
(2, 31, 0, 0.20955109596252441)
(2, 32, 0, 0.06534600257873535)
(2, 33, 0, 0.18881487846374512)
(2, 34, 0, 0.21373367309570312)
(2, 35, 0, 0.21148085594177246)
(2, 36, 0, 0.2096703052520752)
(2, 37, 0, 0.2122650146484375)
(2, 38, 0, 0.21241283416748047)
(2, 39, 0, 0.21612119674682617)
(2, 40, 0, 0.2063310146331787)
(2, 41, 0, 0.17776083946228027)
(2, 42, 0, 0.21943306922912598)
(2, 43, 0, 0.1996018886566162)
(2, 44, 0, 0.2116081714630127)
(2, 45, 0, 0.189405

(2, 270, 0, 0.5165669918060303)
(2, 271, 0, 0.5819041728973389)
(2, 272, 0, 0.5076746940612793)
(2, 273, 0, 0.5697019100189209)
(2, 274, 0, 0.5227978229522705)
(2, 275, 0, 0.551232099533081)
(2, 276, 0, 0.5526678562164307)
(2, 277, 0, 0.539301872253418)
(2, 278, 0, 0.5591559410095215)
(2, 279, 0, 0.6094799041748047)
(2, 280, 0, 0.5914769172668457)
(2, 281, 0, 0.5955338478088379)
(2, 282, 0, 0.19089102745056152)
(2, 283, 0, 0.584022045135498)
(2, 284, 0, 0.5273730754852295)
(2, 285, 0, 0.6170639991760254)
(2, 286, 0, 0.5699279308319092)
(2, 287, 0, 0.5657641887664795)
(2, 288, 0, 0.5137190818786621)
(2, 289, 0, 0.598473072052002)
(2, 290, 0, 0.6069278717041016)
(2, 291, 0, 0.5518310070037842)
(2, 292, 0, 0.5779778957366943)
(2, 293, 0, 0.6380641460418701)
(2, 294, 0, 0.21950531005859375)
(2, 295, 0, 0.6267662048339844)
(2, 296, 0, 0.5470459461212158)
(2, 297, 0, 0.5139927864074707)
(2, 298, 0, 0.590656042098999)
(2, 299, 0, 0.6606409549713135)
(2, 300, 0, 0.6012659072875977)
(2, 301, 0,

(3, 31, 0, 0.20949292182922363)
(3, 32, 0, 0.2064523696899414)
(3, 33, 0, 0.20071721076965332)
(3, 34, 0, 0.17589879035949707)
(3, 35, 0, 0.19758820533752441)
(3, 36, 0, 0.21762919425964355)
(3, 37, 0, 0.07064199447631836)
(3, 38, 0, 0.2281041145324707)
(3, 39, 0, 0.2145841121673584)
(3, 40, 0, 0.19871091842651367)
(3, 41, 0, 0.1980140209197998)
(3, 42, 0, 0.22309374809265137)
(3, 43, 0, 0.19272589683532715)
(3, 44, 0, 0.19872403144836426)
(3, 45, 0, 0.2299818992614746)
(3, 46, 0, 0.22656488418579102)
(3, 47, 0, 0.20769810676574707)
(3, 48, 0, 0.22291088104248047)
(3, 49, 0, 0.18440914154052734)
(3, 50, 0, 0.2016141414642334)
(3, 51, 0, 0.20922088623046875)
(3, 52, 0, 0.22417426109313965)
(3, 53, 0, 0.2060379981994629)
(3, 54, 0, 0.2199411392211914)
(3, 55, 0, 0.21840310096740723)
(3, 56, 0, 0.2259528636932373)
(3, 57, 0, 0.19416308403015137)
(3, 58, 0, 0.20240521430969238)
(3, 59, 0, 0.20618009567260742)
(3, 60, 0, 0.22040295600891113)
(3, 61, 0, 0.2338240146636963)
(3, 62, 0, 0.26574

(3, 287, 0, 0.6677849292755127)
(3, 288, 0, 0.5828981399536133)
(3, 289, 0, 0.6192378997802734)
(3, 290, 0, 0.6211309432983398)
(3, 291, 0, 0.5533497333526611)
(3, 292, 0, 0.6165399551391602)
(3, 293, 0, 0.657865047454834)
(3, 294, 0, 0.6581358909606934)
(3, 295, 0, 0.6691780090332031)
(3, 296, 0, 0.6478040218353271)
(3, 297, 0, 0.653597354888916)
(3, 298, 0, 0.6705367565155029)
(3, 299, 0, 0.694584846496582)
(3, 300, 0, 0.617279052734375)
(3, 301, 0, 0.6801900863647461)
(3, 302, 0, 0.25275373458862305)
(3, 303, 0, 0.6825158596038818)
(3, 304, 0, 0.6044032573699951)
(3, 305, 0, 0.6840429306030273)
(3, 306, 0, 0.6889638900756836)
(3, 307, 0, 0.7213270664215088)
(3, 308, 0, 0.6670610904693604)
(3, 309, 0, 0.6323201656341553)
(3, 310, 0, 0.7169361114501953)
(3, 311, 0, 0.7240390777587891)
(3, 312, 0, 0.6187901496887207)
(3, 313, 0, 0.6294279098510742)
(3, 314, 0, 0.6431629657745361)
(3, 315, 0, 0.7567028999328613)
(3, 316, 0, 0.5730431079864502)
(3, 317, 0, 0.7600882053375244)
(3, 318, 0,

(4, 47, 0, 0.2063910961151123)
(4, 48, 0, 0.2140960693359375)
(4, 49, 0, 0.19881987571716309)
(4, 50, 0, 0.216353178024292)
(4, 51, 0, 0.2079780101776123)
(4, 52, 0, 0.18987107276916504)
(4, 53, 0, 0.03933405876159668)
(4, 54, 0, 0.2010507583618164)
(4, 55, 0, 0.18729615211486816)
(4, 56, 0, 0.03979802131652832)
(4, 57, 0, 0.22571206092834473)
(4, 58, 0, 0.22908997535705566)
(4, 59, 0, 0.1881568431854248)
(4, 60, 0, 0.1819438934326172)
(4, 61, 0, 0.22962284088134766)
(4, 62, 0, 0.25938987731933594)
(4, 63, 0, 0.24130821228027344)
(4, 64, 0, 0.24158716201782227)
(4, 65, 0, 0.2696261405944824)
(4, 66, 0, 0.24280691146850586)
(4, 67, 0, 0.22600102424621582)
(4, 68, 0, 0.2761871814727783)
(4, 69, 0, 0.25966310501098633)
(4, 70, 0, 0.27185797691345215)
(4, 71, 0, 0.20937514305114746)
(4, 72, 0, 0.2825798988342285)
(4, 73, 0, 0.2528219223022461)
(4, 74, 0, 0.26914000511169434)
(4, 75, 0, 0.2537050247192383)
(4, 76, 0, 0.2657811641693115)
(4, 77, 0, 0.2651538848876953)
(4, 78, 0, 0.2753338813

(4, 302, 0, 0.6836459636688232)
(4, 303, 0, 0.6539139747619629)
(4, 304, 0, 0.6274120807647705)
(4, 305, 0, 0.6541218757629395)
(4, 306, 0, 0.6751077175140381)
(4, 307, 0, 0.6873669624328613)
(4, 308, 0, 0.6603090763092041)
(4, 309, 0, 0.6017422676086426)
(4, 310, 0, 0.6499032974243164)
(4, 311, 0, 0.6522958278656006)
(4, 312, 0, 0.6620402336120605)
(4, 313, 0, 0.6786351203918457)
(4, 314, 0, 0.6954948902130127)
(4, 315, 0, 0.6843311786651611)
(4, 316, 0, 0.6511201858520508)
(4, 317, 0, 0.13803315162658691)
(4, 318, 0, 0.6314818859100342)
(4, 319, 0, 0.7320911884307861)
(4, 320, 0, 0.5840749740600586)
(4, 321, 0, 0.5975720882415771)
(4, 322, 0, 0.6263890266418457)
(4, 323, 0, 0.6771149635314941)
(4, 324, 0, 0.595893144607544)
(4, 325, 0, 0.7041161060333252)
(4, 326, 0, 0.6723630428314209)
(4, 327, 0, 0.25443100929260254)
(4, 328, 0, 0.5611350536346436)
(4, 329, 0, 0.7134790420532227)
(4, 330, 0, 0.6905977725982666)
(4, 331, 0, 0.7032430171966553)
(4, 332, 0, 0.6524717807769775)
(4, 333

(5, 60, 0, 0.17072415351867676)
(5, 61, 0, 0.21655726432800293)
(5, 62, 0, 0.21222400665283203)
(5, 63, 0, 0.26683688163757324)
(5, 64, 0, 0.25728273391723633)
(5, 65, 0, 0.2688448429107666)
(5, 66, 0, 0.26825404167175293)
(5, 67, 0, 0.23371577262878418)
(5, 68, 0, 0.23149490356445312)
(5, 69, 0, 0.2637817859649658)
(5, 70, 0, 0.20922207832336426)
(5, 71, 0, 0.27980899810791016)
(5, 72, 0, 0.27756595611572266)
(5, 73, 0, 0.27480483055114746)
(5, 74, 0, 0.2857072353363037)
(5, 75, 0, 0.2309269905090332)
(5, 76, 0, 0.27788281440734863)
(5, 77, 0, 0.2636144161224365)
(5, 78, 0, 0.2828662395477295)
(5, 79, 0, 0.044203996658325195)
(5, 80, 0, 0.23284506797790527)
(5, 81, 0, 0.2706930637359619)
(5, 82, 0, 0.25035905838012695)
(5, 83, 0, 0.28595495223999023)
(5, 84, 0, 0.2706308364868164)
(5, 85, 0, 0.2392411231994629)
(5, 86, 0, 0.22949576377868652)
(5, 87, 0, 0.2721521854400635)
(5, 88, 0, 0.26230502128601074)
(5, 89, 0, 0.23677802085876465)
(5, 90, 0, 0.2164170742034912)
(5, 91, 0, 0.23055

(5, 315, 0, 0.6570632457733154)
(5, 316, 0, 0.26191210746765137)
(5, 317, 0, 0.7474617958068848)
(5, 318, 0, 0.7318120002746582)
(5, 319, 0, 0.2533528804779053)
(5, 320, 0, 0.5776247978210449)
(5, 321, 0, 0.2377629280090332)
(5, 322, 0, 0.6245832443237305)
(5, 323, 0, 0.6989240646362305)
(5, 324, 0, 0.6408059597015381)
(5, 325, 0, 0.674138069152832)
(5, 326, 0, 0.7091629505157471)
(5, 327, 0, 0.7533969879150391)
(5, 328, 0, 0.5797040462493896)
(5, 329, 0, 0.7178220748901367)
(5, 330, 0, 0.6976890563964844)
(5, 331, 0, 0.7111661434173584)
(5, 332, 0, 0.6389777660369873)
(5, 333, 0, 0.7244400978088379)
(5, 334, 0, 0.7415707111358643)
(5, 335, 0, 0.27605104446411133)
(5, 336, 0, 0.7040259838104248)
(5, 337, 0, 0.25629210472106934)
(5, 338, 0, 0.7200682163238525)
(5, 339, 0, 0.7479259967803955)
(5, 340, 0, 0.2837519645690918)
(5, 341, 0, 0.7360329627990723)
(5, 342, 0, 0.7386119365692139)
(5, 343, 0, 0.7530930042266846)
(5, 344, 0, 0.7344009876251221)
(5, 345, 0, 0.7657561302185059)
(5, 34

In [3]:
# verify narrow version
import time

def p2RamanujanCongruenceSeries(n): # Ramanujan Revisted, p. 293
    ans=0
    for a in [1..n+1]:
        d=order(2,a)
        purepart=2^d
        oddpart=a/2^d
        #ord(2,tau(2^n))= (I believe) 3d
        # so ord(2,tau(purepart))=ord(2,tau(2^d)) = 3n
        # so tau(purepart) = 2^(3d)
        # so tau(a) = tau(purepart)*tau(oddpart)=
        # 8^d*tau(oddpart); then we replace tau(oddpart)
        # with sigma(oddpart,k=11) by Ramanujan's congruence
        term=8^d*sigma(oddpart,k=11)
        ans=ans+term*x^a
    return D(ans)

def recipP2(n):
    ans=L(1/p2RamanujanCongruenceSeries(n))
    ans=ans.O(n+1)
    return ans

def p2RecipPowerConstantTerm(k):
    g=recipP2(k)^k
    g=g.O(k+1)
    #print("g:");print(g);print()
    #print("list:")
    gl=g.list()
    #print(gl);print()
    ln=len(gl)
    return gl[ln-2]
    

L=LaurentSeriesRing(QQ,'x', default_prec=2010)

ls=[]
start=time.time()
for a in [1..2000]: #default precision must be > max a.
    c=p2RecipPowerConstantTerm(a)
    o2=order(2,c)
    ds=digitsum(2,a)
    test=o2-3*ds
    finish=time.time()
    print((a,test,finish-start))
    if test!=0:ls=ls+[a]
print()
print("no-count:");print();print(len(ls))
print("bad a's:");print();print(ls)

(1, 0, 0.007245779037475586)
(2, 0, 0.026990890502929688)
(3, 0, 0.051758766174316406)
(4, 0, 0.08152985572814941)
(5, 0, 0.1175076961517334)
(6, 0, 0.17271780967712402)
(7, 0, 0.23305773735046387)
(8, 0, 0.292417049407959)
(9, 0, 0.3507249355316162)
(10, 0, 0.4087669849395752)
(11, 0, 0.46603894233703613)
(12, 0, 0.5245218276977539)
(13, 0, 0.5877189636230469)
(14, 0, 0.6475226879119873)
(15, 0, 0.7075419425964355)
(16, 0, 0.767158031463623)
(17, 0, 0.826589822769165)
(18, 0, 0.8857359886169434)
(19, 0, 0.9441080093383789)
(20, 0, 1.006347894668579)
(21, 0, 1.0803377628326416)
(22, 0, 1.146850824356079)
(23, 0, 1.2140228748321533)
(24, 0, 1.2793307304382324)
(25, 0, 1.3468029499053955)
(26, 0, 1.4193239212036133)
(27, 0, 1.486435890197754)
(28, 0, 1.5523128509521484)
(29, 0, 1.618957757949829)
(30, 0, 1.688032865524292)
(31, 0, 1.7545499801635742)
(32, 0, 1.8214728832244873)
(33, 0, 1.8881700038909912)
(34, 0, 1.9536750316619873)
(35, 0, 2.0190677642822266)
(36, 0, 2.0838568210601807)

(293, 0, 25.838502883911133)
(294, 0, 25.96135663986206)
(295, 0, 26.086062908172607)
(296, 0, 26.206346035003662)
(297, 0, 26.328720808029175)
(298, 0, 26.449214935302734)
(299, 0, 26.57389497756958)
(300, 0, 26.695461750030518)
(301, 0, 26.820485830307007)
(302, 0, 26.947587728500366)
(303, 0, 27.07470178604126)
(304, 0, 27.194791793823242)
(305, 0, 27.32009482383728)
(306, 0, 27.443403005599976)
(307, 0, 27.572665691375732)
(308, 0, 27.699606895446777)
(309, 0, 27.82769203186035)
(310, 0, 27.961647748947144)
(311, 0, 28.093910932540894)
(312, 0, 28.223089933395386)
(313, 0, 28.35796284675598)
(314, 0, 28.492127895355225)
(315, 0, 28.632745027542114)
(316, 0, 28.76628303527832)
(317, 0, 28.90413999557495)
(318, 0, 29.0470290184021)
(319, 0, 29.19278883934021)
(320, 0, 29.320042848587036)
(321, 0, 29.452687740325928)
(322, 0, 29.58565878868103)
(323, 0, 29.72044277191162)
(324, 0, 29.855634927749634)
(325, 0, 29.992666721343994)
(326, 0, 30.128795862197876)
(327, 0, 30.268894910812378

(584, 0, 87.94816470146179)
(585, 0, 88.27118587493896)
(586, 0, 88.59601783752441)
(587, 0, 88.94366693496704)
(588, 0, 89.26795077323914)
(589, 0, 89.6140148639679)
(590, 0, 89.96100091934204)
(591, 0, 90.33365201950073)
(592, 0, 90.64399003982544)
(593, 0, 90.97396087646484)
(594, 0, 91.30418086051941)
(595, 0, 91.6571147441864)
(596, 0, 91.98876190185547)
(597, 0, 92.34318685531616)
(598, 0, 92.69916701316833)
(599, 0, 93.07954573631287)
(600, 0, 93.41349196434021)
(601, 0, 93.76853585243225)
(602, 0, 94.12611985206604)
(603, 0, 94.50503087043762)
(604, 0, 94.86169075965881)
(605, 0, 95.2404408454895)
(606, 0, 95.61790895462036)
(607, 0, 96.02305483818054)
(608, 0, 96.3450517654419)
(609, 0, 96.68483471870422)
(610, 0, 97.02536177635193)
(611, 0, 97.3867928981781)
(612, 0, 97.72484087944031)
(613, 0, 98.08858275413513)
(614, 0, 98.45421695709229)
(615, 0, 98.8412618637085)
(616, 0, 99.18802785873413)
(617, 0, 99.56233382225037)
(618, 0, 99.94629669189453)
(619, 0, 100.3362438678741

(871, 0, 229.05230379104614)
(872, 0, 229.65613389015198)
(873, 0, 230.30598282814026)
(874, 0, 230.9571557044983)
(875, 0, 231.65390992164612)
(876, 0, 232.3071527481079)
(877, 0, 232.99655604362488)
(878, 0, 233.6892328262329)
(879, 0, 234.44195365905762)
(880, 0, 235.0649516582489)
(881, 0, 235.73381876945496)
(882, 0, 236.41154098510742)
(883, 0, 237.12920689582825)
(884, 0, 237.8012728691101)
(885, 0, 238.5125048160553)
(886, 0, 239.2237787246704)
(887, 0, 239.97026586532593)
(888, 0, 240.6419997215271)
(889, 0, 241.35389590263367)
(890, 0, 242.07647371292114)
(891, 0, 242.8545846939087)
(892, 0, 243.59271502494812)
(893, 0, 244.377032995224)
(894, 0, 245.16766166687012)
(895, 0, 246.01709580421448)
(896, 0, 246.600891828537)
(897, 0, 247.22620701789856)
(898, 0, 247.8458969593048)
(899, 0, 248.5032799243927)
(900, 0, 249.1158528327942)
(901, 0, 249.76129984855652)
(902, 0, 250.40912103652954)
(903, 0, 251.0964639186859)
(904, 0, 251.71165680885315)
(905, 0, 252.3984978199005)
(90

(1155, 0, 502.2098128795624)
(1156, 0, 503.2925088405609)
(1157, 0, 504.45903277397156)
(1158, 0, 505.62046790122986)
(1159, 0, 506.8646957874298)
(1160, 0, 507.93094396591187)
(1161, 0, 509.0862216949463)
(1162, 0, 510.2490177154541)
(1163, 0, 511.49238181114197)
(1164, 0, 512.6498579978943)
(1165, 0, 513.9043679237366)
(1166, 0, 515.1721217632294)
(1167, 0, 516.5194149017334)
(1168, 0, 517.609857082367)
(1169, 0, 518.7963967323303)
(1170, 0, 519.9995126724243)
(1171, 0, 521.3007197380066)
(1172, 0, 522.5173189640045)
(1173, 0, 523.8102159500122)
(1174, 0, 525.1127209663391)
(1175, 0, 526.5584888458252)
(1176, 0, 527.7732698917389)
(1177, 0, 529.0771398544312)
(1178, 0, 530.3914659023285)
(1179, 0, 531.7825627326965)
(1180, 0, 533.0772569179535)
(1181, 0, 534.4654908180237)
(1182, 0, 535.8747856616974)
(1183, 0, 537.4046280384064)
(1184, 0, 538.5681190490723)
(1185, 0, 539.8129699230194)
(1186, 0, 541.0735149383545)
(1187, 0, 542.4239349365234)
(1188, 0, 543.7039148807526)
(1189, 0, 5

(1439, 0, 921.7818880081177)
(1440, 0, 923.4625339508057)
(1441, 0, 925.2664928436279)
(1442, 0, 927.0917158126831)
(1443, 0, 929.0478138923645)
(1444, 0, 930.9080367088318)
(1445, 0, 932.9044468402863)
(1446, 0, 934.8885560035706)
(1447, 0, 937.1890518665314)
(1448, 0, 939.0919008255005)
(1449, 0, 941.1381678581238)
(1450, 0, 943.2372601032257)
(1451, 0, 945.3506669998169)
(1452, 0, 947.299567937851)
(1453, 0, 949.3671329021454)
(1454, 0, 951.495080947876)
(1455, 0, 953.67333984375)
(1456, 0, 955.4916088581085)
(1457, 0, 957.5691499710083)
(1458, 0, 959.5819797515869)
(1459, 0, 961.6970047950745)
(1460, 0, 963.7255308628082)
(1461, 0, 965.8267939090729)
(1462, 0, 967.9477498531342)
(1463, 0, 970.2124180793762)
(1464, 0, 972.1755418777466)
(1465, 0, 974.2199308872223)
(1466, 0, 976.2714350223541)
(1467, 0, 978.4159166812897)
(1468, 0, 980.4126839637756)
(1469, 0, 982.5329418182373)
(1470, 0, 984.6773998737335)
(1471, 0, 986.973061800003)
(1472, 0, 988.664500951767)
(1473, 0, 990.502535

(1716, 0, 1521.4340648651123)
(1717, 0, 1524.0694139003754)
(1718, 0, 1526.6971640586853)
(1719, 0, 1529.5133299827576)
(1720, 0, 1532.0429468154907)
(1721, 0, 1534.7570519447327)
(1722, 0, 1537.468894958496)
(1723, 0, 1540.2956018447876)
(1724, 0, 1542.9227666854858)
(1725, 0, 1545.7281668186188)
(1726, 0, 1548.647543668747)
(1727, 0, 1551.6657857894897)
(1728, 0, 1553.8928928375244)
(1729, 0, 1556.2387719154358)
(1730, 0, 1558.6308557987213)
(1731, 0, 1561.2163536548615)
(1732, 0, 1563.6428489685059)
(1733, 0, 1566.241593837738)
(1734, 0, 1568.8313119411469)
(1735, 0, 1571.5259819030762)
(1736, 0, 1573.8846487998962)
(1737, 0, 1576.4081830978394)
(1738, 0, 1578.936482667923)
(1739, 0, 1581.65762591362)
(1740, 0, 1584.2590339183807)
(1741, 0, 1587.0965948104858)
(1742, 0, 1589.9798038005829)
(1743, 0, 1592.8857858181)
(1744, 0, 1595.3416540622711)
(1745, 0, 1597.8962707519531)
(1746, 0, 1600.481281042099)
(1747, 0, 1603.2220396995544)
(1748, 0, 1605.8391358852386)
(1749, 0, 1608.62802

(1993, 0, 2350.0386118888855)
(1994, 0, 2353.4106068611145)
(1995, 0, 2356.9970288276672)
(1996, 0, 2360.3753809928894)
(1997, 0, 2363.9671669006348)
(1998, 0, 2367.5660049915314)
(1999, 0, 2371.3702206611633)
(2000, 0, 2374.535458803177)

no-count:

0
bad a's:

[]
